<a href="https://colab.research.google.com/github/Zasegor/fourier-series-prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import plotly.graph_objects as go
from plotly.io import write_image


---

In [2]:
class FourierForecaster:
    """
    A class for forecasting time series using Fourier series with cosine terms.

    Args:
        l (int): Length of the period.
        m (int): Number of periods.
        p (int): Delay size for 'delay' method or AR order for 'arima' method.
        method (str): Forecasting method ('delay' for delay matrix or 'arima' for ARIMA).
        reg_lambda (float): Regularization parameter for solving linear systems.
    """
    def __init__(self, l: int, m: int, p: int = 1, method: str = "delay", reg_lambda: float = 1e-6):
        self.l = l
        self.m = m
        self.p = p
        self.method = method.lower()
        self.reg_lambda = reg_lambda
        self.matrix = None
        self.new_coefs = None
        self.prediction = None
        self.coefs_for_all_periods = None

        if self.method not in ["delay", "arima"]:
            raise ValueError("Method must be 'delay' or 'arima'")
        if self.l < 1 or self.m < 1 or self.p < 1:
            raise ValueError("l, m, and p must be positive integers")

    def _cos(self, k: int, t: int, l: int) -> float:
        """Computes the cosine term for Fourier series."""
        return math.cos(math.pi * k * t / l)

    def _get_matrix_and_vector(self, period_i: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Constructs the matrix and vector for Fourier coefficient computation."""
        if len(period_i) < 2:
            raise ValueError("period_i must have at least 2 elements")

        l = len(period_i) - 1
        t = np.arange(l + 1)
        k = np.arange(1, l + 1)

        cos_values = np.cos(np.pi * k[:, None] * t / l)
        matrix = np.hstack([0.5 * np.ones((l + 1, 1)), cos_values.T])
        y = period_i.copy()

        return matrix, y

    def _solve_system(self, M: np.ndarray, b: np.ndarray) -> np.ndarray:
        """Solves a linear system with optional regularization."""
        if np.linalg.cond(M) > 1e6:
            M_reg = M + self.reg_lambda * np.eye(M.shape[0])
            return np.linalg.solve(M_reg, b)
        return np.linalg.solve(M, b)

    def _get_matrix_from_series(self, series: pd.Series) -> np.ndarray:
        """Converts the time series into a matrix of period observations."""
        expected_length = self.m * self.l
        if len(series) < expected_length:
            raise ValueError(f"Series length {len(series)} is less than required {expected_length}")

        series_values = series.values[:expected_length]
        if len(series_values) < expected_length:
            series_values = np.pad(series_values, (0, expected_length - len(series_values)), mode='constant')

        return series_values.reshape(self.m, self.l)

    def _get_delay_matrix(self, input_vector: np.ndarray) -> np.ndarray:
        """Constructs a delay matrix for k-NN forecasting."""
        input_vector_copy = np.copy(input_vector)
        m = input_vector_copy.shape[0] % self.p

        if m != 0:
            input_vector_copy = np.delete(input_vector_copy, range(m))

        row_dim = input_vector_copy.shape[0] // self.p
        delay_matrix = np.resize(input_vector_copy, new_shape=(row_dim, self.p)).T

        return delay_matrix

    def _find_nearest(self, row: np.ndarray) -> set:
        """Finds indices of nearest neighbors for k-NN forecasting."""
        neighbors_cnt = 2 * self.p + 1
        last_element = row[-1]
        all_neighbors = row[:-1]
        idx = set(np.argsort(np.abs(all_neighbors - last_element))[:neighbors_cnt])

        return idx

    def _predict_by_one_step(self, input_vector: np.ndarray) -> float:
        """Predicts one step ahead using the specified method."""
        if self.method == "arima":
            if len(input_vector) < 3:
                return input_vector[-1]
            try:
                model = ARIMA(input_vector, order=(min(self.p, 2), 0, 0))
                model_fit = model.fit()
                return model_fit.forecast(steps=1)[0]
            except Exception:
                return input_vector[-1]
        else:  # delay method
            delay_matrix = self._get_delay_matrix(input_vector)
            last_row = delay_matrix[-1, :]
            nearest_neighbors_indexes = self._find_nearest(last_row)

            y = np.empty((0,))
            X = np.empty((0, self.p + 1))
            for index in nearest_neighbors_indexes:
                y = np.append(y, delay_matrix[0, index + 1])
                row = np.append(np.array([1]), delay_matrix[:, index])
                row = np.reshape(row, (1, self.p + 1))
                X = np.append(X, row, axis=0)

            coef = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
            return sum(np.append(np.array([1]), delay_matrix[:, -1]) * coef)

    def _get_new_fourier_coefs(self, periods: np.ndarray) -> list:
        """Computes Fourier coefficients for the next period."""
        new_coefs = []
        coefs_for_all_periods = []

        for period in periods:
            X, y = self._get_matrix_and_vector(period)
            fourier_coef_for_period = self._solve_system(X, y)
            coefs_for_all_periods.append(fourier_coef_for_period)

        coefs_for_all_periods = np.array(coefs_for_all_periods)
        self.coefs_for_all_periods = coefs_for_all_periods

        for i in range(coefs_for_all_periods.shape[1]):
            coef_for_next_period = self._predict_by_one_step(coefs_for_all_periods[:, i])
            new_coefs.append(coef_for_next_period)

        return new_coefs

    def _predict_next_period(self) -> list:
        """Predicts the time series for the next period."""
        new_period = []
        for t in range(self.l):
            s = self.new_coefs[0] / 2
            for k in range(1, len(self.new_coefs)):
                if np.abs(self.new_coefs[k]) > 0.1:
                    s += self.new_coefs[k] * self._cos(k, t, self.l - 1)
            new_period.append(s)

        return new_period

    def fit(self, series: pd.Series) -> 'FourierForecaster':
        """
        Fits the model to the input time series.

        Args:
            series: Input time series (pandas Series).

        Returns:
            Self for method chaining.
        """
        self.matrix = self._get_matrix_from_series(series)
        self.new_coefs = self._get_new_fourier_coefs(self.matrix)
        return self

    def predict(self, index: pd.Index = None) -> pd.Series:
        """
        Predicts the next period.

        Args:
            index: Index for the predicted series (optional).

        Returns:
            Predicted series for the next period (pandas Series).
        """
        self.prediction = self._predict_next_period()
        if index is None:
            index = pd.RangeIndex(start=0, stop=self.l)
        return pd.Series(self.prediction, index=index)

    def evaluate(self, test_series: pd.Series, end_date: str = None) -> tuple[float, float]:
        """
        Evaluates the prediction against a test series.

        Args:
            test_series: Test time series (pandas Series).
            end_date: End date for evaluation (optional, e.g., '2020-02-21').

        Returns:
            Tuple of (MAE, MAPE) metrics.
        """
        if self.prediction is None:
            raise ValueError("Model must be fitted and predicted before evaluation")

        pred = pd.Series(self.prediction[:min(self.l, len(test_series))], index=test_series.index[:min(self.l, len(test_series))])
        if end_date:
            pred = pred[pred.index <= pd.Timestamp(end_date)]
            test = test_series[test_series.index <= pd.Timestamp(end_date)]
        else:
            test = test_series[:self.l]

        mae = round(mean_absolute_error(test, pred), 2)
        mape = round(mean_absolute_percentage_error(test, pred), 3)
        return mae, mape

    def plot(self, test_series: pd.Series, end_date: str = None) -> go.Figure:
        """
        Plots the predicted and test series using Plotly.

        Args:
            test_series: Test time series (pandas Series).
            end_date: End date for plotting (optional, e.g., '2020-02-21').

        Returns:
            Plotly Figure object.
        """
        if self.prediction is None:
            raise ValueError("Model must be fitted and predicted before plotting")

        pred = pd.Series(self.prediction[:min(self.l, len(test_series))], index=test_series.index[:min(self.l, len(test_series))])
        if end_date:
            pred = pred[pred.index <= pd.Timestamp(end_date)]
            test = test_series[test_series.index <= pd.Timestamp(end_date)]
        else:
            test = test_series[:self.l]

        mae, mape = self.evaluate(test_series)

        fig = go.Figure()
        fig.add_scatter(
            x=pred.index,
            y=pred.values,
            line={'width': 2},
            name="Pred"
        )
        fig.add_scatter(
            x=test.index,
            y=test.values,
            line={'width': 2},
            name="Test"
        )
        fig.add_annotation(
            x=0, y=1,
            xref="paper", yref="paper",
            text=f"MAE: {mae}, MAPE: {mape}",
            showarrow=False,
            font=dict(size=18, color="black")
        )
        fig.update_layout(
            title="",
            xaxis_title="Date",
            yaxis_title="Value",
            template="plotly_white"
        )
        return fig

# Синтетические данные

In [ ]:
start_date = '2013-01-01'
dates = pd.date_range(start=start_date, periods=360*10, freq='D')
n_days = len(dates)

np.random.seed(42)
t = np.arange(n_days)

cycle1 = 50 * np.cos(2 * np.pi * t / 180)
cycle2 = 30 * np.cos(2 * np.pi * t / 360)

trend = 0.1 * t + 100

noise = np.random.normal(0, 10, n_days)

prices = trend + cycle1 + cycle2 + noise
prices = np.maximum(prices, 10)

series = pd.Series(prices, index=dates, name='price')

df = pd.DataFrame({'datetime': series.index, 'price': series.values})
df.to_csv('synthetic_prices.csv', index=False)

print(series.head())

2013-01-01    184.967142
2013-01-02    178.682329
2013-01-03    186.536813
2013-01-04    195.215279
2013-01-05    177.498791
Freq: D, Name: price, dtype: float64


In [ ]:
series = pd.Series(df['price'].values, index=pd.to_datetime(df['datetime']))

train_size = 360*9
train_series = series[:train_size]
test_series = series[train_size:]

In [ ]:
fig = go.Figure()

fig.add_scatter(
    x=train_series.index,
    y=train_series.values,
    line={'width':2},
    name="Train"
)

fig.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width':2},
    name="Test"
)

fig.update_layout(
    title='',
    scene=dict(
        xaxis_title='Дата',
        yaxis_title=''
    ),
    showlegend=True,
    height=600
)

In [ ]:
forecaster = FourierForecaster(l=360, m=9, p=2, method="arima")
forecaster.fit(train_series)

test_pred = forecaster.predict(index=test_series.index[:360])

mae, mape = forecaster.evaluate(test_series)
print(f"MAE: {mae}, MAPE: {mape}")

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



MAE: 10.62, MAPE: 0.024


In [ ]:
fig1 = forecaster.plot(test_series)
fig1.show()

In [ ]:
fig.add_scatter(
    x=test_pred.index,
    y=test_pred.values,
    line={'width':2},
    name="Pred"
)

## Компоненты

In [ ]:
coef_data = pd.DataFrame(forecaster.coefs_for_all_periods, columns=range(len(forecaster.coefs_for_all_periods[0])))
coef_data.head()

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,235.496784,-14.754385,28.710754,-1.353438,50.094466,-1.077920,-0.702983,0.572196,-0.331562,0.539147,...,-0.833142,0.400558,0.256612,-0.092436,0.724284,-0.597818,0.352170,1.019434,-0.945112,-0.121423
1,307.003233,-13.778839,30.419860,0.184009,49.826932,-1.376534,1.303770,-0.628828,-0.398769,-0.555920,...,0.648073,0.512762,0.946180,0.048965,0.310528,-0.427519,0.541474,-1.346199,0.866480,-0.403884
2,381.610068,-14.174162,28.453157,-1.804668,50.005057,-0.511427,-1.646439,-0.557553,-0.124418,0.097157,...,-0.136470,1.204083,0.704814,-0.446465,-0.032748,-0.640515,-0.269560,1.224518,-0.606607,0.556807
3,452.857304,-13.678368,30.728197,-1.284397,51.228674,-1.474139,-0.052072,-0.494746,-1.188631,0.265846,...,-0.316601,0.183462,0.317160,-1.316072,-0.396333,1.248639,1.438566,1.436534,0.273143,0.067535
4,525.053494,-13.822991,29.804056,0.747799,49.923181,-1.516457,-0.892048,0.180319,0.190583,-0.118341,...,0.470674,0.538668,0.422222,0.785175,-0.024424,-0.253392,-0.011280,0.092271,0.454291,-0.480168


In [ ]:
# свободный член

fig2 = go.Figure()

fig2.add_scatter(
    x=coef_data.index,
    y=coef_data[0].values,
    line={'width':2},
    name=f"Coef {0}"
)

fig2.add_scatter(
    x=[9],
    y=[forecaster.new_coefs[0]],
    marker={'size':10},
    name=f"Prediction {0}"
)

In [ ]:
# важные компоненты

fig2 = go.Figure()
for i in range(360):
  if i == 1 or i == 2 or i == 4:
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2},
        name=f"Coef {i}"
    )
    fig2.add_scatter(
        x=[9],
        y=[forecaster.new_coefs[i]],
        marker={'size':10},
        name=f"Prediction {i}"
    )
fig2

In [ ]:
# маловажные компоненты

fig2 = go.Figure()
for i in range(360):
  if i != 0 and i != 1 and i != 2 and i != 4:
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2},
        name=f"Coef {i}"
    )
fig2

In [ ]:
# важные компоненты ряда

cos_ = {1:[], 2:[], 4:[]}
new_period = []
for t in range(forecaster.l):
    s = forecaster.new_coefs[0] / 2
    for k in list(cos_.keys()):
        cos_[k].append(forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)+forecaster.new_coefs[0] / 2)
        s += forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)
    new_period.append(s)


fig3 = go.Figure()

fig3.add_scatter(
    x=test_pred.index,
    y=cos_[1],
    line={'width':2},
    name=f"Cos(pi*t/360)"
)

fig3.add_scatter(
    x=test_pred.index,
    y=cos_[2],
    line={'width':2},
    name=f"Cos(2pi*t/360)"
)

fig3.add_scatter(
    x=test_pred.index,
    y=cos_[4],
    line={'width':2},
    name=f"Cos(4pi*t/360)"
)

In [ ]:
# исключение шумов для сглаживания

mae = round(mean_absolute_error(test_series.values, new_period), 2)
mape = round(mean_absolute_percentage_error(test_series.values, new_period), 3)

fig1 = go.Figure()
fig1.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width': 2},
    name="Test"
)
fig1.add_scatter(
    x=test_pred.index,
    y=new_period,
    line={'width':4},
    name=f"Smooth"
)
fig1.add_annotation(
    x=0, y=1,
    xref="paper", yref="paper",
    text=f"MAE: {mae}, MAPE: {mape}",
    showarrow=False,
    font=dict(size=18, color="black")
)
fig1.update_layout(
    title="",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

# Погода

In [ ]:
data = pd.read_csv('/content/DailyDelhiClimateTrain.csv')[['date', 'meantemp']]
data['date'] = pd.to_datetime(data['date'])
test_data = pd.read_csv('/content/DailyDelhiClimateTest.csv')[['date', 'meantemp']]
test_data['date'] = pd.to_datetime(test_data['date'])

temp_series = data.set_index('date')['meantemp']
temp_series = temp_series[~((temp_series.index.month == 2) & (temp_series.index.day == 29))][:-1]
test_series = test_data.set_index('date')['meantemp']
test_series = test_series[~((test_series.index.month == 2) & (test_series.index.day == 29))]

In [ ]:
fig = go.Figure()

fig.add_scatter(
    x=temp_series.index,
    y=temp_series.values,
    line={'width':2},
    name="Train"
)

fig.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width':2},
    name="Test"
)

fig.update_layout(
    title='Предсказание погоды',
    scene=dict(
        xaxis_title='Дата',
        yaxis_title='oC'
    ),
    showlegend=True,
    height=600
)

In [ ]:
m = 4
l = 365
p = 1

index = pd.date_range("2017-01-01", "2017-12-31", freq="D")

forecaster = FourierForecaster(l=l, m=m, p=p, method="arima")
forecaster.fit(temp_series)

test_pred = forecaster.predict(index=index)

mae, mape = forecaster.evaluate(test_series)
print(f"MAE: {mae}, MAPE: {mape}")

fig1 = forecaster.plot(test_series)
fig1.show()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_ret

MAE: 2.56, MAPE: 0.118


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [ ]:
coef_data = pd.DataFrame(forecaster.coefs_for_all_periods, columns=range(len(forecaster.coefs_for_all_periods[0])))
coef_data.head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,49.651897,-0.908617,-9.519335,-1.089227,-2.624976,1.077334,0.149410,-0.797769,-0.162273,-0.146470,...,0.122167,0.098553,0.105261,0.050628,0.065951,-0.019224,-0.011945,0.119018,0.060774,0.032516
1,50.088025,-1.544720,-9.877819,-0.166875,-1.898749,1.434172,-0.314710,0.072762,0.160513,0.578533,...,0.030556,-0.028652,0.025600,0.103459,0.028016,-0.014683,0.065013,-0.101242,0.023845,-0.083296
2,50.285444,-1.348398,-9.188799,-0.485232,-2.645125,1.530743,0.027974,-0.653853,-0.577953,0.041382,...,0.108817,0.046379,-0.130379,-0.006419,0.006824,0.055722,-0.037942,-0.040786,0.127627,0.048334
3,54.291627,-0.919987,-8.533939,-1.549562,-3.169410,1.278360,-0.013334,-0.226957,0.175303,0.194263,...,0.002784,-0.156686,-0.022052,0.004123,0.066005,0.046288,0.053314,0.146678,-0.028222,-0.030341


In [ ]:
0, 1, 2, 4, 5

(0, 1, 2, 4, 5)

In [ ]:
fig.add_scatter(x=test_pred.index,
                y=test_pred.values,
                line={'width':2},
                name="Pred"
)

## Компоненты

In [ ]:
coef_data = pd.DataFrame(forecaster.coefs_for_all_periods, columns=range(len(forecaster.coefs_for_all_periods[0])))
coef_data.head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,49.651897,-0.908617,-9.519335,-1.089227,-2.624976,1.077334,0.149410,-0.797769,-0.162273,-0.146470,...,0.122167,0.098553,0.105261,0.050628,0.065951,-0.019224,-0.011945,0.119018,0.060774,0.032516
1,50.088025,-1.544720,-9.877819,-0.166875,-1.898749,1.434172,-0.314710,0.072762,0.160513,0.578533,...,0.030556,-0.028652,0.025600,0.103459,0.028016,-0.014683,0.065013,-0.101242,0.023845,-0.083296
2,50.285444,-1.348398,-9.188799,-0.485232,-2.645125,1.530743,0.027974,-0.653853,-0.577953,0.041382,...,0.108817,0.046379,-0.130379,-0.006419,0.006824,0.055722,-0.037942,-0.040786,0.127627,0.048334
3,54.291627,-0.919987,-8.533939,-1.549562,-3.169410,1.278360,-0.013334,-0.226957,0.175303,0.194263,...,0.002784,-0.156686,-0.022052,0.004123,0.066005,0.046288,0.053314,0.146678,-0.028222,-0.030341


In [ ]:
# свободный член

fig2 = go.Figure()

fig2.add_scatter(
    x=coef_data.index,
    y=coef_data[0].values,
    line={'width':2},
    name=f"Coef {0}"
)

fig2.add_scatter(
    x=[4],
    y=[forecaster.new_coefs[0]],
    marker={'size':10},
    name=f"Prediction {0}"
)

In [ ]:
# важные компоненты

colors = {1:'red', 2:'teal', 3:'gold', 4:'blue', 5:'yellowgreen'}

fig2 = go.Figure()
for i in range(365):
  if i == 1 or i == 2 or i == 3 or i == 4 or i == 5:
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2, 'color':colors[i]},
        name=f"Coef {i}"
    )
    fig2.add_scatter(
        x=[4],
        y=[forecaster.new_coefs[i]],
        marker={'size':10, 'color':colors[i]},
        name=f"Prediction {i}"
    )
fig2

In [ ]:
# маловажные компоненты

fig2 = go.Figure()
for i in range(1, 365):
  if not (i == 1 or i == 2 or i == 3 or i == 4 or i == 5):
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2},
        name=f"Coef {i}"
    )
fig2

In [ ]:
# важные компоненты ряда

cos_ = {1:[], 2:[], 3:[], 4:[], 5:[]}
fig3 = go.Figure()
new_period = []
for t in range(forecaster.l):
    s = forecaster.new_coefs[0] / 2
    for k in list(cos_.keys()):
        cos_[k].append(forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)+forecaster.new_coefs[0] / 2)
        s += forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)
    new_period.append(s)


for k in list(cos_.keys()):
    fig3.add_scatter(
        x=test_pred.index,
        y=cos_[k],
        line={'width':2},
        name=f"Cos({k}pi*t/360)"
    )

fig3

In [ ]:
# исключение шумов для сглаживания

mae = round(mean_absolute_error(test_series.values, new_period[:len(test_series)]), 2)
mape = round(mean_absolute_percentage_error(test_series.values, new_period[:len(test_series)]), 3)

fig1 = go.Figure()
fig1.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width': 2},
    name="Test"
)
fig1.add_scatter(
    x=test_pred.index[:len(test_series)],
    y=new_period[:len(test_series)],
    line={'width':4},
    name=f"Smooth"
)
fig1.add_annotation(
    x=0, y=1,
    xref="paper", yref="paper",
    text=f"MAE: {mae}, MAPE: {mape}",
    showarrow=False,
    font=dict(size=18, color="black")
)
fig1.update_layout(
    title="",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

---

# Самолеты

In [3]:
data = pd.read_csv('/content/euro_arrivals.csv')
data['FLT_DATE'] = pd.to_datetime(data['FLT_DATE'])

data_series = data.set_index('FLT_DATE')['FLT_ARR_1']
data_series = data_series[~((data_series.index.month == 2) & (data_series.index.day == 29))]

In [4]:
[len(data_series[data_series.index.year == i]) for i in range(2016, 2025)]

[365, 365, 365, 365, 365, 365, 365, 365, 365]

In [5]:
fig = go.Figure()

fig.add_scatter(
    x=data_series.index,
    y=data_series.values,
    line={'width':2},
    name="Data"
)

fig.update_layout(
    title='',
    scene=dict(
        xaxis_title='',
        yaxis_title=''
    ),
    showlegend=True,
    height=600
)

In [6]:
train_series = data_series[data_series.index.year<=2019]
test_series = data_series[data_series.index.year==2020]
test_series = test_series[test_series.index<=pd.Timestamp('2020-02-21')]

In [7]:
test_series.head()

,FLT_ARR_1
FLT_DATE,
2020-01-01,16056
2020-01-02,20778
2020-01-03,21282
2020-01-04,18666
2020-01-05,20757


In [8]:
fig = go.Figure()

fig.add_scatter(
    x=train_series.index,
    y=train_series.values,
    line={'width':2},
    name="Train"
)

fig.update_layout(
    title='',
    scene=dict(
        xaxis_title='',
        yaxis_title=''
    ),
    showlegend=True,
    height=600
)

In [9]:
m = 4
l = 365
p = 1

index = pd.date_range(start='2020-01-01', end='2020-12-31', freq='D')
index = index[~((index.month == 2) & (index.day == 29))]

forecaster = FourierForecaster(l=l, m=m, p=p)
forecaster.fit(train_series)

test_pred = forecaster.predict(index=index)

mae, mape = forecaster.evaluate(test_series, end_date='2020-02-21')
print(f"MAE: {mae}, MAPE: {mape}")

fig1 = forecaster.plot(test_series, end_date='2020-02-21')
fig1.show()

MAE: 1224.14, MAPE: 0.062


In [10]:
forecaster_arima = FourierForecaster(l=l, m=m, p=p, method="arima")
forecaster_arima.fit(train_series)

test_pred_arima = forecaster_arima.predict(index=index)

mae, mape = forecaster_arima.evaluate(test_series, end_date='2020-02-21')
print(f"MAE: {mae}, MAPE: {mape}")

fig1 = forecaster_arima.plot(test_series, end_date='2020-02-21')
fig1.show()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.



MAE: 1334.79, MAPE: 0.068


In [11]:
fig.add_scatter(
    x=test_pred.index,
    y=test_pred.values,
    line={'width':2},
    name="Pred"
)

fig.add_scatter(
    x=test_pred_arima.index,
    y=test_pred_arima.values,
    line={'width':2},
    name="PredArima"
)

fig.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width':2},
    name="Test"
)

## Компоненты

In [12]:
coef_data = pd.DataFrame(forecaster.coefs_for_all_periods, columns=range(len(forecaster.coefs_for_all_periods[0])))
coef_data.head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,41744.535714,-1148.839095,-3223.456786,725.789998,-488.611926,233.162799,-5.610470,-264.327860,163.729806,-73.300063,...,12.148917,-33.619760,-27.288557,-53.471019,-6.304882,-39.702831,28.410786,-24.037763,33.295965,-18.012363
1,43335.057692,-1098.706500,-3499.916992,851.545926,-533.115082,280.526024,3.813606,-200.333723,240.782946,-75.971194,...,4.906601,-35.894450,-5.407776,-40.005445,-27.483093,-33.734289,-11.950333,-35.197776,0.992435,-26.168956
2,46682.752747,-1315.395706,-3553.258684,833.268001,-516.653017,288.792023,19.957965,-155.499207,346.039494,-34.908537,...,-4.587910,-48.455818,16.905528,-15.827063,40.539064,-9.852279,19.655975,-40.411488,20.263745,-36.892857
3,47134.148352,-914.411599,-3635.330687,928.302600,-431.077578,289.232906,11.784484,-282.133467,323.382333,-52.172000,...,4.769200,-22.247478,-23.988546,-54.557411,5.164571,-40.350840,23.833942,-27.852245,7.051352,-29.865385


In [13]:
# свободный член

fig2 = go.Figure()

fig2.add_scatter(
    x=coef_data.index,
    y=coef_data[0].values,
    line={'width':2},
    name=f"Coef {0}"
)

fig2.add_scatter(
    x=[4],
    y=[forecaster.new_coefs[0]],
    marker={'size':10},
    name=f"Prediction {0}"
)

In [40]:
# важные компоненты

colors = {1:'red', 2:'teal', 3:'gold', 4:'blue', 208:'yellowgreen', 104:'pink'}

fig2 = go.Figure()
for i in range(365):
  if i == 1 or i == 2 or i == 3 or i == 4 or i == 208 or i == 104:
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2, 'color':colors[i]},
        name=f"Coef {i}"
    )
    fig2.add_scatter(
        x=[4],
        y=[forecaster.new_coefs[i]],
        marker={'size':10, 'color':colors[i]},
        name=f"Prediction {i}"
    )
fig2

In [34]:
# маловажные компоненты

fig2 = go.Figure()
for i in range(1, 365):
  if not (i == 1 or i == 2 or i == 3 or i == 4 or i == 208 or i == 104):
    fig2.add_scatter(
        x=coef_data.index,
        y=coef_data[i].values,
        line={'width':2},
        name=f"Coef {i}"
    )
fig2

In [47]:
# важные компоненты ряда

cos_ = {1:[], 2:[], 3:[], 4:[], 104:[]}
fig3 = go.Figure()
new_period = []
for t in range(forecaster.l):
    s = forecaster.new_coefs[0] / 2
    for k in list(cos_.keys()):
        cos_[k].append(forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)+forecaster.new_coefs[0] / 2)
        s += forecaster.new_coefs[k] * forecaster._cos(k, t, forecaster.l - 1)
    new_period.append(s)


for k in list(cos_.keys()):
    fig3.add_scatter(
        x=test_pred.index,
        y=cos_[k],
        line={'width':2},
        name=f"Cos({k}pi*t/360)"
    )

fig3

In [48]:
# исключение шумов для сглаживания

mae = round(mean_absolute_error(test_series.values, new_period[:len(test_series)]), 2)
mape = round(mean_absolute_percentage_error(test_series.values, new_period[:len(test_series)]), 3)

fig1 = go.Figure()
fig1.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width': 2},
    name="Test"
)
fig1.add_scatter(
    x=test_pred.index[:len(test_series)],
    y=new_period[:len(test_series)],
    line={'width':4},
    name=f"Smooth"
)
fig1.add_annotation(
    x=0, y=1,
    xref="paper", yref="paper",
    text=f"MAE: {mae}, MAPE: {mape}",
    showarrow=False,
    font=dict(size=18, color="black")
)
fig1.update_layout(
    title="",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

---

# SNP

In [ ]:
data  = pd.read_csv("/content/SNP/Download Data - INDEX_US_S&P US_SPX (0).csv", parse_dates=['Date'], index_col='Date')
for i in range(1, 10):
  tempdata = pd.read_csv(f"/content/SNP/Download Data - INDEX_US_S&P US_SPX ({i}).csv", parse_dates=['Date'], index_col='Date')
  data = data.combine_first(tempdata)

data.head()

,Open,High,Low,Close
Date,,,,
2016-01-04,"2,038.20","2,038.20","1,989.68","2,012.66"
2016-01-05,"2,013.78","2,021.94","2,004.17","2,016.71"
2016-01-06,"2,011.71","2,011.71","1,979.05","1,990.26"
2016-01-07,"1,985.32","1,985.32","1,938.83","1,943.09"
2016-01-08,"1,945.97","1,960.40","1,918.46","1,922.03"


In [ ]:
data = pd.read_csv("/content/snp500_history.csv", parse_dates=['Date'], index_col='Date')
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0


In [ ]:
data = data[data.index>pd.Timestamp('2010-01-01 00:00:00-05:00')]

In [ ]:
data = pd.read_csv("/content/snpdataset.csv", parse_dates=['timestamp'], index_col='timestamp')
data.head()

,open,high,low,close,volume
timestamp,,,,,
2017-09-11 09:30:00,44.01,44.05,44.0100,44.01,NaN
2017-09-11 09:31:00,44.01,44.25,44.0000,44.25,139460.0
2017-09-11 09:32:00,44.25,44.32,44.2200,44.27,53119.0
2017-09-11 09:33:00,44.28,44.46,44.2504,44.34,38228.0
2017-09-11 09:34:00,44.34,44.36,44.2200,44.24,32831.0


In [ ]:
data.index[-1]

Timestamp('2017-11-07 11:12:00')

In [ ]:
data.describe()

,open,high,low,close,volume
count,16119.000000,16119.000000,16119.000000,16119.000000,15666.000000
mean,48.876498,48.897112,48.855891,48.876059,3469.346036
std,2.532328,2.532869,2.531208,2.531823,5713.668844
min,44.010000,44.050000,44.000000,44.010000,100.000000
25%,47.115000,47.130000,47.100000,47.115000,1000.000000
50%,47.710000,47.730000,47.690000,47.709900,2100.000000
75%,51.340000,51.370000,51.300000,51.334700,3900.000000
max,53.640000,53.740000,53.470000,53.630000,297047.000000


In [ ]:
fig = go.Figure()

fig.add_scatter(
    x=data.index,
    y=data['close'],
    line={'width':2},
    name="Data"
)

In [ ]:
fig = go.Figure()

fig.add_scatter(
    x=train_series.index,
    y=train_series.values,
    line={'width':2},
    name="Train"
)

fig.add_scatter(
    x=test_series.index,
    y=test_series.values,
    line={'width':2},
    name="Test"
)

fig.update_layout(
    title='Предсказание погоды',
    scene=dict(
        xaxis_title='Дата',
        yaxis_title='oC'
    ),
    showlegend=True,
    height=600
)